In [1]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [3]:
df = pd.read_csv("../../diaries_transactions_all.csv", dtype={'account_startclose_balance': str})

In [4]:
dfb = df[(df["trx_family_code"]=="FRMLN")&((df["trx_type_code"]==2760)|(df["trx_type_code"]==3247))]

In [8]:
dfb["trx_prx_purpose"].unique()

array(['6. Closing Balance--End of last DQ',
       '1. Starting balance (today)', '5. Interest accruing',
       '3. Payments', 'INCREASE--Unexplained balance adjustment',
       '2. New borrowinng (individual portion)', '4. Any known fees',
       'DECREASE--Unexplained balance adjustment',
       '7. Refund from lender', '2. New borrowing'], dtype=object)

In [9]:
for acc in dfb["account_ids"].unique():
    u = 0
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        if row["trx_prx_purpose"]=="DECREASE--Unexplained balance adjustment" or row["trx_prx_purpose"]=="INCREASE--Unexplained balance adjustment":
            u = 1
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        dfb.at[index,"acc_unexplained_trx"] = u

In [10]:
s = set()
for acc in dfb[dfb["acc_unexplained_trx"]==0]["account_ids"].unique():
     for i in dfb[dfb["account_ids"]==acc]["trx_prx_purpose"].unique():
            s.add(i)
print(s)

{'4. Any known fees', '2. New borrowinng (individual portion)', '7. Refund from lender', '2. New borrowing', '1. Starting balance (today)', '6. Closing Balance--End of last DQ', '5. Interest accruing', '3. Payments'}


In [11]:
dfb = dfb[dfb["acc_unexplained_trx"]==0]

In [27]:
del dfb["new_account_ids"]
dfb.insert(df.columns.get_loc("account_ids")+1,"new_account_ids", ["-"]*(dfb.shape[0]))
dfb.sample(1)

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
288513,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,25jan2013,1,2013,...,07=Group meeting,Outflow,12350.0,145.2941,0,NaN,NaN,NaN,5.689655,0.0


### Here I "cleaned" data for housing in excel. What can be extracted: if owned, number and value of additional property 

In [56]:
df1 = pd.read_csv('../housing_conditions_ownership.csv')
df1.replace({"yes":1, "no":0}, inplace=True)
dfh=df1[['hh_ids','owned','N', 'other_prop_or_shamba_value' ]]
dfh.head()

,hh_ids,owned,N,other_prop_or_shamba_value
0,KNBOK31,0,NaN,NaN
1,KNBOK01,1,NaN,NaN
2,KNBOK02,1,NaN,NaN
3,KNBOK04,1,1.0,1500000.0
4,KNBOK05,1,NaN,NaN


In [57]:
house=pd.merge(dfb, dfh, on = 'hh_ids', how='outer')


In [58]:
house.to_csv('../datasets/house.csv')